### Paso 1: Análisis preliminar de los datos y diferencia de medias

En esta primera parte, se importan los datos contenidos en el archivo `A3.1 Khan.csv`, el cual contiene información de expresión génica para 2308 genes y una variable de clase (`y`) que indica el tipo de cáncer (valores del 1 al 4).

Se revisa que no haya valores faltantes, y posteriormente se comparan las medias de expresión génica entre las clases **2 y 4**. Se calcula la diferencia de medias para cada gen, y se imprimen los **10 genes con mayor diferencia absoluta** entre estas dos clases.

Este tipo de análisis puede ser útil en estudios de inferencia, ya que una gran diferencia de medias en la expresión génica entre dos tipos de cáncer podría indicar que ese gen está diferencialmente expresado, y por lo tanto podría estar relacionado con mecanismos biológicos distintos o incluso ser un posible biomarcador diagnóstico.


In [3]:
import pandas as pd
import numpy as np

# Cargar los datos
data = pd.read_csv('A3.1 Khan.csv')

# Verificar si hay valores faltantes
print("¿Hay valores faltantes?", data.isnull().values.any())

# Separar X (todas las columnas excepto 'y') e y (la clase)
X = data.drop(columns=['y'])
y = data['y']

# Filtrar las clases 2 y 4
X_2 = X[y == 2]
X_4 = X[y == 4]

# Calcular diferencia de medias
diff_means = X_2.mean() - X_4.mean()
top_genes = diff_means.abs().sort_values(ascending=False).head(10)

# Mostrar los 10 genes con mayor diferencia de medias
print("Top 10 genes con mayor diferencia de medias entre clases 2 y 4:")
print(top_genes)


¿Hay valores faltantes? False
Top 10 genes con mayor diferencia de medias entre clases 2 y 4:
X187     3.323151
X509     2.906537
X2046    2.424515
X2050    2.401783
X129     2.165185
X1645    2.065460
X1319    2.045941
X1955    2.037340
X1003    2.011337
X246     1.837830
dtype: float64


### Paso 2: Pruebas t y corrección por pruebas múltiples

En este paso se comparan las medias de expresión génica entre las clases 2 y 4 utilizando pruebas **t independientes** para cada uno de los genes. Como se están realizando miles de pruebas al mismo tiempo (una por gen), es necesario corregir los valores p para evitar un alto número de falsos positivos.

Se utilizan tres métodos de corrección por pruebas múltiples:

- **Bonferroni**: muy conservador, divide el nivel de significancia entre el número de pruebas.
- **Holm**: también controla el error tipo I pero es menos conservador que Bonferroni.
- **Benjamini-Hochberg (FDR)**: controla la tasa de falsos descubrimientos, más adecuado cuando se espera que muchas pruebas puedan ser significativas.

Se imprimen cuántos genes resultan significativamente distintos entre las clases 2 y 4 bajo cada método, utilizando un umbral de significancia de 0.05.


In [4]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

# Realizar prueba t para cada gen entre clases 2 y 4
p_values = []
t_stats = []

for gene in X.columns:
    t_stat, p_val = ttest_ind(X_2[gene], X_4[gene], equal_var=False)
    t_stats.append(t_stat)
    p_values.append(p_val)

# Convertir a arrays
p_values = np.array(p_values)
t_stats = np.array(t_stats)

# Corrección de p-values
# Bonferroni
reject_bonf, pvals_bonf, _, _ = multipletests(p_values, alpha=0.05, method='bonferroni')
# Holm
reject_holm, pvals_holm, _, _ = multipletests(p_values, alpha=0.05, method='holm')
# Benjamini-Hochberg (FDR)
reject_bh, pvals_bh, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

# Mostrar cuántos genes fueron significativos con cada método
print("Genes significativos (Bonferroni):", np.sum(reject_bonf))
print("Genes significativos (Holm):", np.sum(reject_holm))
print("Genes significativos (Benjamini-Hochberg):", np.sum(reject_bh))


Genes significativos (Bonferroni): 72
Genes significativos (Holm): 72
Genes significativos (Benjamini-Hochberg): 296


### Paso 3: ANOVA para comparar expresión génica entre las 4 clases

En este paso se realiza un análisis de varianza (ANOVA de un factor) para cada uno de los genes con el fin de determinar si existen diferencias significativas en la expresión génica entre las 4 clases (tipos de cáncer).

Se utiliza la función `f_oneway` de `scipy.stats`, que compara las medias de varios grupos y devuelve un valor p para cada gen. Posteriormente, se aplican los mismos métodos de corrección por pruebas múltiples usados en el paso anterior:

- Bonferroni
- Holm
- Benjamini-Hochberg (FDR)

Finalmente, se reporta cuántos genes presentan diferencias estadísticamente significativas en al menos una de las clases, según cada método de corrección.


In [5]:
from scipy.stats import f_oneway

# Separar los datos por clase
X_1 = X[y == 1]
X_2 = X[y == 2]
X_3 = X[y == 3]
X_4 = X[y == 4]

# Prueba ANOVA para cada gen
anova_p_values = []

for gene in X.columns:
    f_stat, p_val = f_oneway(X_1[gene], X_2[gene], X_3[gene], X_4[gene])
    anova_p_values.append(p_val)

anova_p_values = np.array(anova_p_values)

# Correcciones por múltiples pruebas
reject_bonf, pvals_bonf, _, _ = multipletests(anova_p_values, alpha=0.05, method='bonferroni')
reject_holm, pvals_holm, _, _ = multipletests(anova_p_values, alpha=0.05, method='holm')
reject_bh, pvals_bh, _, _ = multipletests(anova_p_values, alpha=0.05, method='fdr_bh')

# Mostrar cuántos genes fueron significativos
print("Genes significativos (Bonferroni):", np.sum(reject_bonf))
print("Genes significativos (Holm):", np.sum(reject_holm))
print("Genes significativos (Benjamini-Hochberg):", np.sum(reject_bh))


Genes significativos (Bonferroni): 404
Genes significativos (Holm): 412
Genes significativos (Benjamini-Hochberg): 1162


### Paso 4: Modelos SVM con distintos kernels

En este paso se entrenan modelos de Máquinas de Vectores de Soporte (SVM) para clasificar los tipos de cáncer en función de la expresión génica. Para reducir el tiempo de cómputo, se seleccionan los 20 genes más significativos según el análisis ANOVA corregido por Benjamini-Hochberg en el paso anterior.

Los modelos entrenados son:

- **SVM con kernel lineal**: busca separar las clases con un hiperplano.
- **SVM con kernel polinomial (grado 3)**: transforma los datos a un espacio no lineal de mayor dimensión.
- **SVM con kernel RBF (Radial Basis Function)**: modelo no lineal que usa funciones de base radial para separar clases.

Los datos se dividen en conjunto de entrenamiento (70%) y prueba (30%), manteniendo la proporción de clases con `stratify=y`. Se reportan las métricas de clasificación para cada modelo, lo que permitirá comparar su desempeño en el siguiente paso.


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Usaremos los 20 genes más significativos según ANOVA (Benjamini-Hochberg)
top_gene_indices = np.argsort(anova_p_values)[:20]
top_gene_names = X.columns[top_gene_indices]

# Filtrar X con solo esos genes
X_reduced = X[top_gene_names]

# Separar datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=42, stratify=y)

# SVM con kernel lineal
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)
y_pred_linear = svm_linear.predict(X_test)
print("SVM con kernel lineal:")
print(classification_report(y_test, y_pred_linear))

# SVM con kernel polinomial grado 3
svm_poly = SVC(kernel='poly', degree=3)
svm_poly.fit(X_train, y_train)
y_pred_poly = svm_poly.predict(X_test)
print("SVM con kernel polinomial (grado 3):")
print(classification_report(y_test, y_pred_poly))

# SVM con kernel radial (RBF)
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)
y_pred_rbf = svm_rbf.predict(X_test)
print("SVM con kernel RBF:")
print(classification_report(y_test, y_pred_rbf))


SVM con kernel lineal:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00         8

    accuracy                           1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25

SVM con kernel polinomial (grado 3):
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      0.89      0.94         9
           3       1.00      1.00      1.00         5
           4       0.89      1.00      0.94         8

    accuracy                           0.96        25
   macro avg       0.97      0.97      0.97        25
weighted avg       0.96      0.96      0.96        25

SVM con kernel RBF:
              precision    recall  f1-score   supp

### Paso 5: Comparación de modelos SVM

A continuación se comparan los resultados de los tres modelos de SVM entrenados:

- **Kernel lineal**: obtuvo un desempeño perfecto en el conjunto de prueba, clasificando correctamente todas las muestras. Esto sugiere que los datos, al menos al reducirse a los 20 genes más relevantes, son linealmente separables.
  
- **Kernel polinomial (grado 3)**: aunque logró un alto desempeño general, cometió errores leves en la predicción de las clases 2 y 4. Esto puede deberse a una transformación innecesaria o incluso perjudicial del espacio de características, dado que el kernel lineal ya era suficiente.

- **Kernel RBF (Radial)**: también obtuvo una precisión perfecta, similar al modelo lineal. Este tipo de kernel es más flexible y puede adaptarse a fronteras más complejas, aunque en este caso no ofreció una ventaja adicional.

**Conclusión:** tanto el modelo con kernel **lineal** como el de **RBF** funcionaron muy bien, pero el lineal podría considerarse preferible por su menor complejidad y alta interpretabilidad. Esto indica que, al menos con esta selección de genes, el problema puede resolverse efectivamente con un modelo lineal.
